In [96]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Concatenate
from tensorflow.keras import Model
from keras import backend as K
from tensorflow.keras.metrics import AUC
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
import tensorflow_addons as tfa

In [97]:
#Train dataset creation
num_steps = 24
num_features_t = 35
timeseries_train_df = pd.read_csv('/media/nasim/31c299f0-f952-4032-9bd8-001b141183e0/ML-Libraries-Graph-Database-Neo4j/PyG-Neo4j/app/eICU_data/train/timeseries.csv')
flat_train_df = pd.read_csv('/media/nasim/31c299f0-f952-4032-9bd8-001b141183e0/ML-Libraries-Graph-Database-Neo4j/PyG-Neo4j/app/eICU_data/train/flat.csv')
labels_train_df = pd.read_csv('/media/nasim/31c299f0-f952-4032-9bd8-001b141183e0/ML-Libraries-Graph-Database-Neo4j/PyG-Neo4j/app/eICU_data/train/labels.csv')
arr_timeseries_train = timeseries_train_df.values
x_train_reshape = np.reshape(arr_timeseries_train, (-1, num_steps, num_features_t))
x_train_reshape = x_train_reshape[:, :, 1:] # Removed patient column

flat_train_df = flat_train_df.fillna(flat_train_df.mean())
x_train_over_static = flat_train_df.iloc[:, 1:]

y_train = labels_train_df['actualiculos']

In [98]:
#Test dataset creation
num_steps = 24
num_features_t = 35
timeseries_test_df = pd.read_csv('/media/nasim/31c299f0-f952-4032-9bd8-001b141183e0/ML-Libraries-Graph-Database-Neo4j/PyG-Neo4j/app/eICU_data/test/timeseries.csv')
flat_test_df = pd.read_csv('/media/nasim/31c299f0-f952-4032-9bd8-001b141183e0/ML-Libraries-Graph-Database-Neo4j/PyG-Neo4j/app/eICU_data/test/flat.csv')
labels_test_df = pd.read_csv('/media/nasim/31c299f0-f952-4032-9bd8-001b141183e0/ML-Libraries-Graph-Database-Neo4j/PyG-Neo4j/app/eICU_data/test/labels.csv')
arr_timeseries_test = timeseries_test_df.values
x_test_reshape = np.reshape(arr_timeseries_test, (-1, num_steps, num_features_t))
x_test_reshape = x_test_reshape[:, :, 1:] # Removed patient column

flat_test_df = flat_test_df.fillna(flat_train_df.mean())
x_test_over_static = flat_test_df.iloc[:, 1:]

y_test = labels_test_df['actualiculos']

In [99]:
#Validation dataset creation
num_steps = 24
num_features_t = 35
timeseries_val_df = pd.read_csv('/media/nasim/31c299f0-f952-4032-9bd8-001b141183e0/ML-Libraries-Graph-Database-Neo4j/PyG-Neo4j/app/eICU_data/val/timeseries.csv')
flat_val_df = pd.read_csv('/media/nasim/31c299f0-f952-4032-9bd8-001b141183e0/ML-Libraries-Graph-Database-Neo4j/PyG-Neo4j/app/eICU_data/val/flat.csv')
labels_val_df = pd.read_csv('/media/nasim/31c299f0-f952-4032-9bd8-001b141183e0/ML-Libraries-Graph-Database-Neo4j/PyG-Neo4j/app/eICU_data/val/labels.csv')
arr_timeseries_val = timeseries_val_df.values
x_val_reshape = np.reshape(arr_timeseries_val, (-1, num_steps, num_features_t))
x_val_reshape = x_val_reshape[:, :, 1:] # Removed patient column

flat_val_df = flat_val_df.fillna(flat_train_df.mean())
x_val_over_static = flat_val_df.iloc[:, 1:]

y_val = labels_val_df['actualiculos']

In [100]:
# Define timesteps and the number of featuresn_timesteps = 24
n_timesteps = 24
n_features = 34
n_outputs = 1
epochs = 10
batch_size = 64
verbose = False

In [101]:
# Define input layer
recurrent_input = Input(shape=(n_timesteps,n_features),name="TIMESERIES_INPUT")

static_input = Input(shape=(x_train_over_static.shape[1], ),name="STATIC_INPUT")

In [102]:
# layer - 1
rec_layer_one = Bidirectional(LSTM(128, kernel_regularizer=regularizers.L2(0.01), recurrent_regularizer=regularizers.L2(0.01),return_sequences=True),name ="BIDIRECTIONAL_LAYER_1")(recurrent_input)
rec_layer_one = Dropout(0.1,name ="DROPOUT_LAYER_1")(rec_layer_one)
# layer - 2
rec_layer_two = Bidirectional(LSTM(64, kernel_regularizer=regularizers.L2(0.01), recurrent_regularizer=regularizers.L2(0.01)),name ="BIDIRECTIONAL_LAYER_2")(rec_layer_one)
rec_layer_two = Dropout(0.1,name ="DROPOUT_LAYER_2")(rec_layer_two)
# SLP Layers
static_layer_one = Dense(64,  kernel_regularizer=regularizers.L2(0.001),  activation='relu',name="DENSE_LAYER_1")(static_input)

In [103]:
# Combine layers - RNN + SLP
combined = Concatenate(axis= 1,name = "CONCATENATED_TIMESERIES_STATIC")([rec_layer_two,static_layer_one])
combined_dense_two = Dense(64, activation='relu',name="DENSE_LAYER_2")(combined)
output = Dense(n_outputs,activation='linear',name="OUTPUT_LAYER")(combined_dense_two)

In [104]:
# Compile ModeL
model = Model(inputs=[recurrent_input,static_input],outputs=[output])

In [105]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = K.cast(y_true, y_pred.dtype)
    diff = K.abs((y_true - y_pred) / K.clip(K.abs(y_true),
                                            K.epsilon(),
                                            None))
    return 100. * K.mean(diff, axis=-1)

In [106]:
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [107]:
# binary cross entropy loss
model.compile(loss='mse', optimizer='adam', metrics=[mean_absolute_percentage_error, coeff_determination])

In [108]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 TIMESERIES_INPUT (InputLayer)  [(None, 24, 34)]     0           []                               
                                                                                                  
 BIDIRECTIONAL_LAYER_1 (Bidirec  (None, 24, 256)     166912      ['TIMESERIES_INPUT[0][0]']       
 tional)                                                                                          
                                                                                                  
 DROPOUT_LAYER_1 (Dropout)      (None, 24, 256)      0           ['BIDIRECTIONAL_LAYER_1[0][0]']  
                                                                                                  
 BIDIRECTIONAL_LAYER_2 (Bidirec  (None, 128)         164352      ['DROPOUT_LAYER_1[0][0]']  

In [ ]:
history = model.fit([np.asarray(x_train_reshape).astype('float32'), np.asarray(x_train_over_static).astype('float32')], y_train, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data=([np.asarray(x_val_reshape).astype('float32'), np.asarray(x_val_over_static).astype('float32')], y_val))

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict([np.asarray(x_test_reshape).astype('float32'),np.asarray(x_test_over_static).astype('float32')])
y_pred

In [ ]:
output = model.evaluate([np.asarray(x_test_reshape).astype('float32'),np.asarray(x_test_over_static).astype('float32')], y_test, batch_size=batch_size, verbose=0)
output

In [ ]:
print("MAPE:{}, R2:{}".format(output[1], output[2])